In [9]:
from src.constants import *
import os
from src.config.entity import *
from src.loggingInfo.loggingInfo import logging
import gdown
from src.utils.utils import read_yaml, create_directories

In [13]:
import logging
import sys
import zipfile

from src.exception.exception import customException


class ConfigurationManger:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config_filePath = config_file_path
        self.params_filePath = params_file_path
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
    
    def get_data_ingestion_config(self):
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return(data_ingestion_config)
    

class DataIngestion:
    def __init__(self, config: ConfigurationManger):
        self.config = config
    
    def download_file(self):
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logging.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logging.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise customException(e, sys)
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise customException(e, sys)

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1lMpB7PvFW8TVyjeDqfXQIW59qp9EIC5B
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1lMpB7PvFW8TVyjeDqfXQIW59qp9EIC5B&confirm=t&uuid=e570250b-2537-493e-aadd-82ed3bfa06d8
To: d:\FarmconProject\plantClassificationPart\artifacts\data_ingestion\data.zip
100%|██████████| 3.51G/3.51G [17:38<00:00, 3.32MB/s] 
